In [1]:
import mediapipe as mp
import cv2
import time
import os

In [2]:
mp_face_mesh = mp.solutions.face_mesh

**Eye Height**

In [3]:
import math 
def are_eyes_closed(landmarks, image):
    coords=[]
    for i in (386, 362, 374, 263, 159, 33, 145, 133):
        x2_loc=int(landmarks.landmark[i].x*image.shape[1])
#         print(x2_loc)
        y2_loc=int(landmarks.landmark[i].y*image.shape[0])
        coords.append((x2_loc,y2_loc))
#Coords go right eye, left eye, top, left, bottom, right
#         print(coords)
        
    sum=0
    right_width = math.sqrt( (coords[1][0] - coords[3][0])**2 +
                          (coords[1][1] - coords[3][1])**2   )
    left_width = math.sqrt( (coords[5][0] - coords[7][0])**2 +
                          (coords[5][1] - coords[7][1])**2   )
    
    # distance between two near points up and down
    right_distance = math.sqrt( (coords[0][0] - coords[2][0])**2 +
                              (coords[0][1] - coords[2][1])**2   )
    left_distance = math.sqrt( (coords[4][0] - coords[6][0])**2 +
                              (coords[4][1] - coords[6][1])**2   )
    
#     print("right_width="+str(right_width))
#     print("right_dist="+str(right_distance))
#     print("left_width="+str(left_width))
#     print("left_dist="+str(left_distance))
    ratio=0.25
    if right_distance < right_width * ratio and left_distance < left_width * ratio:
#         print("Closed")
        return True
    else:
        return False

# are_eyes_closed(results.face_landmarks, frame)

**Lip Height**

In [4]:
(0, 13, 14, 17, 37, 82, 87, 84, 267, 312, 317, 314)
#top_lip_top, top_lip_bottom, bottom_lip_top, bottom_lip_bottom

(0, 13, 14, 17, 37, 82, 87, 84, 267, 312, 317, 314)

In [5]:
def get_lip_height(landmarks, image):
    coords=[]
    for i in (0, 13, 14, 17, 37, 82, 87, 84, 267, 312, 317, 314):
        x2_loc=int(landmarks.landmark[i].x*image.shape[1])
#         print(x2_loc)
        y2_loc=int(landmarks.landmark[i].y*image.shape[0])
        coords.append((x2_loc,y2_loc))

    sum_top=0
    sum_bottom=0
    #0,4,8
    for i in [0,4,8]:
        # distance between two near points up and down
        top_distance = math.sqrt( (coords[i][0] - coords[1+i][0])**2 +
                              (coords[i][1] - coords[1+i][1])**2   )
    for i in [2,6,10]:
        # distance between two near points up and down
        bottom_distance = math.sqrt( (coords[i][0] - coords[1+i][0])**2 +
                              (coords[i][1] - coords[1+i][1])**2   )

    sum_top += top_distance
    sum_bottom += bottom_distance
    return (sum_top / 3,sum_bottom / 3)

# top_lip, bottom_lip=get_lip_height(results.face_landmarks, frame)
# print(top_lip)
# print(bottom_lip)

In [6]:
def get_mouth_height(landmarks, image):
    coords=[]
    for i in (0, 13, 14, 17, 37, 82, 87, 84, 267, 312, 317, 314):
        x2_loc=int(landmarks.landmark[i].x*image.shape[1])
#         print(x2_loc)
        y2_loc=int(landmarks.landmark[i].y*image.shape[0])
        coords.append((x2_loc,y2_loc))
    
    sum=0
    for i in [1,5,9]:
        # distance between two near points up and down
        distance = math.sqrt( (coords[i][0] - coords[i+1][0])**2 + 
                              (coords[i][1] - coords[i+1][1])**2   )
        sum += distance
    return sum / 3

# get_mouth_height(results.face_landmarks, frame)

In [7]:
def check_mouth_open(landmarks, image):
    top_lip_height, bottom_lip_height=get_lip_height(landmarks, image)
    mouth_height=get_mouth_height(landmarks, image)

#     print(min(top_lip_height, bottom_lip_height)*0.5)
#     print(mouth_height)
    
    # if mouth is open more than lip height * ratio, return true.
    ratio = 1.0
    if mouth_height > min(top_lip_height, bottom_lip_height) * ratio:
        return True
    else:
        return False
    
# check_mouth_open(results.face_landmarks, frame)

In [8]:
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, 
                                  min_detection_confidence=0.5, min_tracking_confidence=0.5)

def Eyes_and_mouth(frame):
    results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    if results.multi_face_landmarks is not None:
        if len(results.multi_face_landmarks[0].landmark) > 0:
            closed_eyes=are_eyes_closed(results.multi_face_landmarks[0], frame)

            open_mouth=check_mouth_open(results.multi_face_landmarks[0], frame)

            if closed_eyes:
                cv2.putText(frame, 'Eyes Closed', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA )
            elif open_mouth:
                cv2.putText(frame, 'Mouth Open', (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA )

**Live video**

In [10]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

f = 0
previous=time.time()

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('mouth_output.mp4', fourcc, 24, (640, 480)) 

while cap.isOpened():
    new = time.time()
    f = int(1/(new - previous))
    previous = new
      
    ret, frame = cap.read()
    if not ret:
        break
        
    Eyes_and_mouth(frame)
        
    cv2.putText(frame, f'FPS: {f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1, cv2.LINE_AA )
    cv2.imshow('Webcam Testing Landmarks', frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()